In [1]:
import pandas as pd
import Levenshtein

In [2]:
words = pd.read_csv('words.csv')

In [3]:
words.head()

,Français,Hiragana,Kanji
0,enseignant,きょうし,教室
1,étudiant,がくせい,学生
2,employé d'une compagnie,かいしゃいん,会社員
3,employé (avec nom de compagnie),しゃいん,社員
4,employé de banque,ぎんこういん,銀行員


In [4]:
shuffled_words = words.sample(frac=1).reset_index(drop=True)

In [5]:
def is_similar_japanese_answer(user_answer, correct_answer):
    similarity_threshold = 0.8
    distance = Levenshtein.distance(user_answer.lower(), correct_answer.lower())
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    return similarity >= similarity_threshold

In [6]:
def is_similar_french_answer(user_answer, correct_answer):
    similarity_threshold = 0.5
    if pd.isna(user_answer) or pd.isna(correct_answer):
        return False
    user_answer = user_answer.lower()
    correct_answer = correct_answer.lower()
    substring_match = user_answer in correct_answer or correct_answer in user_answer
    if substring_match:
        return True
    distance = Levenshtein.distance(user_answer, correct_answer)
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    
    return similarity >= similarity_threshold

In [7]:
shuffled_sample = shuffled_words.sample(n=20)
shuffled_sample

,Français,Hiragana,Kanji
143,problème,もんだい,問題
128,express,きゅうこう,急行
145,beaux-arts,びじゅつ,美術
158,emprunter,かります,借ります
134,petite monnaie,こまかいおかね,細かいお金
81,université,だいがく,大学
28,rendre / rembourser,かえします,返します
79,gentil / aimable,しんせつ,親切
117,contemplation fleurs de cerisiers,おはなみ,お花見
37,s'amuser / jouer,あそびます,遊びます


In [10]:
seen_fr = []
correct_count = 0
score_fr = 0
shuffled_sample_jp = shuffled_sample.sample(frac=1).reset_index(drop=True)

for i in range(len(shuffled_sample_jp)):
    if shuffled_sample_jp['Français'].iloc[i] not in seen_fr:
        ask_h = input(f"Comment dire {shuffled_sample_jp['Français'].iloc[i]} en hiragana ?")
        seen_fr.append(shuffled_sample_jp['Français'].iloc[i])

        if is_similar_japanese_answer(ask_h, shuffled_sample_jp['Hiragana'].iloc[i]):
            kanji_value = shuffled_sample_jp['Kanji'].iloc[i]
            if pd.notna(kanji_value):
                ask_k = input("Et en kanji ?")
                if is_similar_japanese_answer(ask_k, kanji_value):
                    correct_count += 1
                    score_fr += 1
                    print('Correct !')
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
                else:
                    print(f"Incorrect ! La bonne réponse est : {kanji_value}")
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
            else:
                correct_count += 1
                score_fr += 1
                print('Correct !')
                print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
        else:
            print(f"Incorrect ! La bonne réponse est : {shuffled_sample_jp['Hiragana'].iloc[i]}")
            print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')

if correct_count == len(shuffled_sample_jp):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Comment dire réserver en hiragana ?よやくします
Et en kanji ?予約します
Correct !
Bonnes réponses : 1/20
Comment dire problème en hiragana ?もんだい
Et en kanji ?問題
Correct !
Bonnes réponses : 2/20
Comment dire étagère en hiragana ?たな
Et en kanji ?棚
Correct !
Bonnes réponses : 3/20
Comment dire mandarine en hiragana ?
Incorrect ! La bonne réponse est : みかん
Bonnes réponses : 3/20
Comment dire se reposer / prendre un congé en hiragana ?やすみます
Et en kanji ?休みます
Correct !
Bonnes réponses : 4/20
Comment dire rendre / rembourser en hiragana ?かえます
Et en kanji ?換えます
Incorrect ! La bonne réponse est : 返します
Bonnes réponses : 4/20
Comment dire accueillir / aller chercher en hiragana ?むかえります
Et en kanji ?迎えります
Correct !
Bonnes réponses : 5/20
Comment dire petite monnaie en hiragana ?おかんきん
Incorrect ! La bonne réponse est : こまかいおかね
Bonnes réponses : 5/20
Comment dire gentil / aimable en hiragana ?しんせつ
Et en kanji ?親切
Correct !
Bonnes réponses : 6/20
Comment dire s'amuser / jouer en hiragana ?あそびます
Et en kanji ?遊びま

In [11]:
seen_k=[]
correct_count = 0
score_jp = 0
shuffled_sample_fr = shuffled_sample.sample(frac=1).reset_index(drop=True)
for i in range(len(shuffled_sample_fr)):
    kanji = shuffled_sample_fr['Kanji'].iloc[i]
    if pd.notna(kanji) and kanji not in seen_k:
        ask_h = input(f"Que veut dire ce kanji : {kanji}?")
        if is_similar_french_answer(ask_h, shuffled_sample_fr['Français'].iloc[i]):
            seen_k.append(kanji)
            correct_count += 1
            score_jp += 1
            print(f"Correct ! C'était bien {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')
        else:
            print(f"Incorrecte ! La bonne réponse était : {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')

if correct_count == len(shuffled_sample_fr):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Que veut dire ce kanji : 細かいお金?petite monnaie
Correct ! C'était bien petite monnaie
Bonnes réponses : 1/20
Que veut dire ce kanji : 親切?gentil
Correct ! C'était bien gentil / aimable
Bonnes réponses : 2/20
Que veut dire ce kanji : 予約します?réserver
Correct ! C'était bien réserver
Bonnes réponses : 3/20
Que veut dire ce kanji : 開けます?ouvrir
Correct ! C'était bien ouvrir
Bonnes réponses : 4/20
Que veut dire ce kanji : 休みます?prendre un congès
Incorrecte ! La bonne réponse était : se reposer / prendre un congé
Bonnes réponses : 4/20
Que veut dire ce kanji : 遊びます?jouer
Correct ! C'était bien s'amuser / jouer
Bonnes réponses : 5/20
Que veut dire ce kanji : 迎えります?accueillir
Correct ! C'était bien accueillir / aller chercher
Bonnes réponses : 6/20
Que veut dire ce kanji : 紅茶?thé anglais
Correct ! C'était bien thé anglais
Bonnes réponses : 7/20
Que veut dire ce kanji : 返します?rembourser
Correct ! C'était bien rendre / rembourser
Bonnes réponses : 8/20
Que veut dire ce kanji : 独身?célibataire
Correct ! C